In [1]:
from datasets import Dataset
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/clean/sustainability-report-2020-clean-BST_KK-fixed-with_starts.csv', sep=';')

In [3]:
# Randomly copy half of the rows to a new dataframe
data_sample = data.sample(frac=0.5, random_state=1)
data_sample = data_sample.reset_index(drop=True)
# Set all values in the fLABEL column to NOTOK
data_sample['fLABEL'] = 'NOT_OK'
# Shuffle the context column
data_sample['context'] = np.random.permutation(data_sample['context'])
# Get starting positions
for i in range(len(data_sample)):
    text = data_sample["context"][i]
    answer = data_sample["answer"][i]
    answer_start = text.find(answer)
    data_sample.loc[i, "answer_start"] = int(answer_start)

# Set label to OK for all rows where the answer_start is not -1
data_sample.loc[data_sample['answer_start'] != -1, 'fLABEL'] = 'OK'

# Concatenate the original dataframe with the modified sample
data = pd.concat([data, data_sample], ignore_index=True)

In [4]:
# Create a new column containing dictionaries
data['answers'] = data.apply(lambda x: {'text': x['answer'], 'answer_start': [x['answer_start']]}, axis=1)
data["id"] = data.index

In [5]:
data.reset_index(drop=True, inplace=True)

In [6]:
train_df = data.sample(frac=0.8, random_state=1)
test_df = data.drop(train_df.index)

In [7]:
train_ds = Dataset.from_pandas(train_df, split="train")
test_ds = Dataset.from_pandas(test_df, split="test")

In [8]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [10]:
train_dataset = train_ds.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_ds.column_names,
)
len(train_ds), len(train_dataset)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

(222, 222)

In [11]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]
    
    inputs["example_id"] = example_ids
    return inputs

In [12]:
validation_dataset = test_ds.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_ds.column_names,
)
len(test_ds), len(validation_dataset)

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

(55, 55)

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [16]:
trainer.train()

c:\Users\rjutr\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.2512, 'eval_samples_per_second': 218.924, 'eval_steps_per_second': 15.922, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.2536, 'eval_samples_per_second': 216.854, 'eval_steps_per_second': 15.771, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.2512, 'eval_samples_per_second': 218.925, 'eval_steps_per_second': 15.922, 'epoch': 3.0}
{'train_runtime': 12.7208, 'train_samples_per_second': 52.355, 'train_steps_per_second': 3.302, 'train_loss': 0.9919716971261161, 'epoch': 3.0}


TrainOutput(global_step=42, training_loss=0.9919716971261161, metrics={'train_runtime': 12.7208, 'train_samples_per_second': 52.355, 'train_steps_per_second': 3.302, 'train_loss': 0.9919716971261161, 'epoch': 3.0})